# Касьяненко Вера

# Символьное дифференцирование

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 18 ноября 10:00**

Символьное дифференцирование это инструмент для автоматического вывода формул производных, который открывает возможности для анализа сложных функций, оптимизации процессов и работы с уравнениями. Мы уже на многих занятиях сталкивались с этой темой - давайте попробуем реализовать собственное!

## Выражение

Создадим основной класс `Expr`, от которого будут наследоваться различные типы выражений, такие как константы, переменные, суммы, произведения и другие. Класс должен содержать методы:
* `__call__`, который будет вычислять значение выражения, используя переданный ему контекст (словарь, связывающий имена переменных с их значениями).
* `d`, принимающий имя переменной, по которой требуется вычислить производную, и возвращающий выражение, представляющее производную по этой переменной.

Эти методы нужно будет переопределить в каждом из подклассов для корректного выполнения операций.

In [1]:
import operator

class Expr:
    def __call__(self, **context):
        raise NotImplementedError("Метод __call__ должен быть реализован в подклассе.")

    def d(self, wrt):
        raise NotImplementedError("Метод d должен быть реализован в подклассе.")

    def __add__(self, other):
        return Sum(self, other)

    def __radd__(self, other):
        return Sum(Const(other) if isinstance(other, (int, float)) else other, self)

    def __sub__(self, other):
        return Sum(self, Product(Const(-1), other))

    def __rsub__(self, other):
        return Sum(other, Product(Const(-1), self))

    def __mul__(self, other):
        return Product(self, other)

    def __rmul__(self, other):
        return Product(Const(other) if isinstance(other, (int, float)) else other, self)

    def __truediv__(self, other):
        return Fraction(self, other)

    def __rtruediv__(self, other):
        return Fraction(other, self)

    def __neg__(self):
        return Product(Const(-1), self)

    def __pos__(self):
        return self

    def __pow__(self, power):
        return Power(self, Const(power))

    def __repr__(self):
        return self.__str__()

Создайте классы для двух видов выражений: `Const`, представляющий константу, и` Var`, представляющий переменную. Чтобы упростить использование, вместо обращения к конструкторам этих классов, будем использовать их однобуквенные сокращённые обозначения.

**Пример использования:**
```python
V = Var
C = Const

C(5)()
5
C(5).d(V("x"))()
0
V("x")(x=5)
5
V("x").d(V("y"))(x=5)
0
V("x").d(V("x"))(x=5)
1
```

In [2]:
class Const(Expr):
    def __init__(self, value):
        self.value = value

    def __call__(self, **context):
        return self.value

    def d(self, wrt):
        return Const(0)

    def __str__(self):
        return str(self.value)

class Var(Expr):
    def __init__(self, name):
        self.name = name

    def __call__(self, **context):
        if self.name in context:
            return context[self.name]
        else:
            raise ValueError(f"Значение переменной '{self.name}' не предоставлено в контексте.")

    def d(self, wrt):
        if self.name == wrt.name:
            return Const(1)
        else:
            return Const(0)

    def __str__(self):
        return self.name

V = Var
C = Const

In [3]:
print(C(5)()) # 5
print(C(5).d(V("x"))()) # 0
print(V("x")(x=5)) # 5
print(V("x").d(V("y"))()) # 0
print(V("x").d(V("x"))()) # 1

5
0
5
0
1


In [24]:
import unittest

class TestConst(unittest.TestCase):
    def test_const_call(self):
        c = Const(5)
        self.assertEqual(c(), 5)
        self.assertEqual(c(x=10), 5)
        self.assertEqual(c(y=3), 5)

    def test_const_derivative(self):
        c = Const(5)
        derivative = c.d(V("x"))
        self.assertIsInstance(derivative, Const)
        self.assertEqual(derivative(), 0)

In [25]:
class TestVar(unittest.TestCase):
    def test_var_call(self):
        x = V("x")
        self.assertEqual(x(x=5), 5)
        self.assertEqual(x(x=10), 10)
        with self.assertRaises(ValueError):
            x()

    def test_var_derivative_self(self):
        x = V("x")
        derivative = x.d(V("x"))
        self.assertIsInstance(derivative, Const)
        self.assertEqual(derivative(), 1)

    def test_var_derivative_other(self):
        x = V("x")
        y = V("y")
        derivative = x.d(y)
        self.assertIsInstance(derivative, Const)
        self.assertEqual(derivative(), 0)

## Бинарные операции

Создайте классы для бинарных операций: `Sum`, `Product` и `Fraction`. Поскольку бинарные операции определяются двумя операндами, их конструктор будет одинаковым для всех этих классов. Поэтому его можно вынести в отдельный базовый класс, чтобы избежать дублирования кода.

In [4]:
class BinOp(Expr):
    def __init__(self, expr1, expr2):
        self.expr1, self.expr2 = expr1, expr2

Реализуйте `Sum` для суммирования, `Product` для умножения и `Fraction` для деления.

**Пример использования:**

```python
Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)
7.0
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)
3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)
-3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)
-1.25
```

In [5]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt):
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def op_symbol(self):
        return "+"

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)

    def d(self, wrt):
        return Sum(Product(self.expr1.d(wrt), self.expr2), Product(self.expr1, self.expr2.d(wrt)))

    def op_symbol(self):
        return "*"

class Fraction(BinOp):
    def __call__(self, **context):
        denominator = self.expr2(**context)
        if denominator == 0:
            raise ZeroDivisionError("Деление на ноль.")
        return self.expr1(**context) / denominator

    def d(self, wrt):
        numerator = Sum(Product(self.expr1.d(wrt), self.expr2), Product(Const(-1), Product(self.expr1, self.expr2.d(wrt))))
        denominator = Product(self.expr2, self.expr2)
        return Fraction(numerator, denominator)

    def op_symbol(self):
        return "/"

In [8]:
print(Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)) # 7.0
print(Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)) # 3.5
print(Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)) # -3.5
print(Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)) # -1.25

7.0
3.5
-3.5
-1.25


In [26]:
class TestSum(unittest.TestCase):
    def test_sum_call(self):
        expr = Sum(C(5), V("x"))
        self.assertEqual(expr(x=3), 8)
        self.assertEqual(expr(x=-2), 3)

    def test_sum_derivative(self):
        expr = Sum(V("x"), C(5))
        derivative = expr.d(V("x"))
        self.assertIsInstance(derivative, Sum)
        self.assertEqual(derivative(x=10), 1)

In [27]:
class TestProduct(unittest.TestCase):
    def test_product_call(self):
        expr = Product(C(5), V("x"))
        self.assertEqual(expr(x=3), 15)
        self.assertEqual(expr(x=-2), -10)

    def test_product_derivative(self):
        expr = Product(V("x"), V("x"))  # f(x) = x * x
        derivative = expr.d(V("x"))
        self.assertIsInstance(derivative, Sum)
        self.assertEqual(derivative(x=3), 6)

In [28]:
class TestFraction(unittest.TestCase):
    def test_fraction_call(self):
        expr = Fraction(V("x"), V("y"))
        self.assertEqual(expr(x=5, y=2.5), 2.0)
        self.assertEqual(expr(x=10, y=2), 5.0)
        with self.assertRaises(ZeroDivisionError):
            expr(x=1, y=0)

    def test_fraction_derivative(self):
        expr = Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))
        derivative_x = expr.d(V("x"))
        self.assertIsInstance(derivative_x, Fraction)
        self.assertAlmostEqual(derivative_x(x=1, y=2), -3.5, places=5)

        derivative_y = expr.d(V("y"))
        self.assertIsInstance(derivative_y, Fraction)
        self.assertAlmostEqual(derivative_y(x=1, y=2), -1.25, places=5)

## Перегрузка операторов

Добавьте перегрузку операторов в базовых класс `Expr`. Обратите что в классах мы можем тоже заменить на использование операторов.
```python  
-e         e.__neg__()
+e         e.__pos__()
e1 + e2    e1.__add__(e2)
e1 - e2    e1.__sub__(e2)
e1 * e2    e1.__mul__(e2)
e1 / e2    e1.__truediv__(e2)
```

**Пример использования:**

```python
(V("x") * V("x") / V("y"))(x=5, y=2.5)
10.0
```

In [9]:
(V("x") * V("x") / V("y"))(x=5, y=2.5)

10.0

## Метод Ньютона-Рафсона

Напишите функцию `newton_raphson`, которая принимает дифференцируемую функцию  $f$  от переменной  $x$ , начальное приближение  $x_0$ , и положительное число  $\epsilon$ , задающее точность вычислений. Функция должна возвращать значение  $x$ , при котором  $f(x)$  становится равным нулю. Метод Ньютона-Рафсона выполняет итеративный поиск корня функции  $f(x)$ , начиная с начального значения  $x_0$ , и использует правило  
$$x_{n+1} = x_n - \frac{f(x_n)}{f{\prime}(x_n)}$$  
для обновления  $x$  на каждом шаге. Итерации продолжаются до тех пор, пока условие остановки  $|x_{n+1} - x_n| \leq \epsilon$  не будет выполнено.

**Пример использования:**

```python
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)
(1.000000000001132, -2.490496697760136e-11)
```

In [22]:
def newton_raphson(expr, x0, eps=1e-4, max_iter=1000):
    x_current = x0
    for i in range(max_iter):
        f_value = expr(x=x_current)
        f_derivative = expr.d(V("x"))(x=x_current)

        if f_derivative == 0:
            raise ZeroDivisionError("Производная равна нулю")

        x_next = x_current - (f_value / f_derivative)

        if abs(x_next - x_current) <= eps:
            return x_next

        x_current = x_next

    raise ValueError("Метод не сошелся за заданное количество итераций")

In [21]:
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)

(1.0000000000000653, -1.4384049507043528e-12)

In [30]:
class TestNewtonRaphson(unittest.TestCase):
    def test_newton_raphson_simple_root(self):
        # Функция f(x) = x - 2, корень x=2
        x = V("x")
        f = Sum(x, C(-2))
        root = newton_raphson(f, x0=0.0, eps=1e-6)
        self.assertAlmostEqual(root, 2.0, places=6)
        self.assertAlmostEqual(f(x=root), 0.0, places=6)

    def test_newton_raphson_polynomial(self):
        # Функция f(x) = x^2 - 4, корни x=2 и x=-2
        x = V("x")
        f = Sum(Product(x, x), C(-4))
        root = newton_raphson(f, x0=3.0, eps=1e-6)
        self.assertAlmostEqual(root, 2.0, places=6)
        self.assertAlmostEqual(f(x=root), 0.0, places=6)

        root_neg = newton_raphson(f, x0=-3.0, eps=1e-6)
        self.assertAlmostEqual(root_neg, -2.0, places=6)
        self.assertAlmostEqual(f(x=root_neg), 0.0, places=6)

    def test_newton_raphson_no_convergence(self):
        # Функция f(x) = x^3 - 2x + 2, не имеет действительных корней
        x = V("x")
        f = Sum(Product(Product(x, x), x), Sum(Product(C(-2), x), C(2)))
        with self.assertRaises(ValueError):
            newton_raphson(f, x0=0.0, eps=1e-6, max_iter=10)

    def test_newton_raphson_zero_derivative(self):
        # Функция f(x) = x^3, производная f'(x) = 3x^2
        x = V("x")
        f = Product(Product(x, x), x)
        with self.assertRaises(ZeroDivisionError):
            newton_raphson(f, x0=0.0, eps=1e-6)

In [31]:
unittest.main(argv=[''], verbosity=2, exit=False)
None

test_const_call (__main__.TestConst) ... ok
test_const_derivative (__main__.TestConst) ... ok
test_fraction_call (__main__.TestFraction) ... ok
test_fraction_derivative (__main__.TestFraction) ... ok
test_newton_raphson_no_convergence (__main__.TestNewtonRaphson) ... ok
test_newton_raphson_polynomial (__main__.TestNewtonRaphson) ... ok
test_newton_raphson_simple_root (__main__.TestNewtonRaphson) ... ok
test_newton_raphson_zero_derivative (__main__.TestNewtonRaphson) ... ok
test_product_call (__main__.TestProduct) ... ok
test_product_derivative (__main__.TestProduct) ... ok
test_sum_call (__main__.TestSum) ... ok
test_sum_derivative (__main__.TestSum) ... ok
test_const_call (__main__.TestSymbolicDifferentiation) ... ok
test_const_derivative (__main__.TestSymbolicDifferentiation) ... ok
test_fraction_call (__main__.TestSymbolicDifferentiation) ... ok
test_fraction_derivative (__main__.TestSymbolicDifferentiation) ... ok
test_newton_raphson_no_convergence (__main__.TestSymbolicDifferentia